# Chapter 6: Decision Trees

## Exercise 1
What is the approximate depth of a Decision Tree trained (without restrictions) on a training set with 1 million instances?

Decision Trees tend to be balanced, so the depth will be: $log_2 (10^6) \approx 20 $

## Exercise 2
Is a node's Gini impurity generally lower or greater than its parent's? Is it generally lower/greater, or always lower/greater?

A node's Gini impurity is *generally* lower than its parent's. A node will not have children if their weighted impurity is greater than that of the father. However, one of the children can have greater impurity than its parent if the other children compensates it by having a low impurity.

## Exercise 3
If a Decision Tree is overfitting the training set, is it a good idea to try decreasing max_depth?

It is a good idea to decrease max_depth.

## Exercise 4
If a Decision Tree is underfitting the training set, is it a good idea to try scaling the input features?

Scaling the input features does not have an effect in Decision Trees, so it is not a good idea to try to scale the input features.

## Exercise 5
If it takes one hour to train a Decision Tree on a training set containing 1 million instances, roughly how much time will it take to train another Decision Tree on a training set containing 10 million instances?

The complexity of training a Decision Tree is $O(m * n * log(m))$, so if we increase the number of instances by 10 times we have that:<br>
$O(10m * n * log(10m)) / O(m * n * log(m)) \approx 11.7$<br><br>
So the training time will take 11.7 times more to complete. In this case, it will take roughly 11h and 42min.

## Exercise 6
If your training set contains 100,000 instances, will setting presort=True speed up training?

Presorting the training set speeds up training only if the dataset is smaller than a few thousand instances. If it contains 100,000 instances, setting presort to True will slow down training.

## Exercise 7
Train and fine-tune a Decision Tree for the moons dataset.

a) Generate a moons dataset using make_moons(n_samples=1000, noise=0.4).

In [1]:
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=10000, noise=0.4)

b) Split it into a training set and a test set using train_test_split().

In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

c) Use grid search with cross_validation to find good hyperparameters for a DecisionTreeClassifier.

In [3]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

params = {
    'criterion': ('gini', 'entropy'),
    'max_depth': [2, 5, 10, 20, 50],
    'min_samples_split': [2, 5, 10],
    'min_weight_fraction_leaf': [0.005, 0.01, 0.05, 0.1, 0.2]
}


clf = DecisionTreeClassifier()
grid = GridSearchCV(clf, params)
grid.fit(X_train, y_train)
grid.best_estimator_

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.005, presort=False,
            random_state=None, splitter='best')

d) Train it on the full training set using these hyperparameters, and measure your model's performance on the test set. You should get roughly 85% to 87% accuracy.

In [4]:
from sklearn.metrics import accuracy_score

y_pred = grid.predict(X_test)
accuracy = accuracy_score(y_pred, y_test)

print('Obtained Accuracy of: {0:.2f}'.format(accuracy))

Obtained Accuracy of: 0.85


## Exercise 8
Grow a forest.

a) Continuing the previous exercise, generate 1,000 subsets of the training set, each containing 100 instances selected randomly.

In [5]:
from sklearn.model_selection import ShuffleSplit

n_trees = 1000
n_instances = 100

mini_sets = []

rs = ShuffleSplit(n_splits=n_trees, test_size=len(X_train) - n_instances, random_state=42)
for mini_train_index, mini_test_index in rs.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))

b) Train one Decision Tree on each subset, using the best hyperparameter values found above. Evaluate these 1,000 Decision Trees on the test set. Since they were trained on smaller sets, these Decision Trees will likely perform worse than the first Decision Tree, archieving only about 80% accuracy.

In [6]:
from sklearn.base import clone
import numpy as np

forest = [clone(grid.best_estimator_) for _ in range(n_trees)]

accuracy_scores = []

for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train, y_mini_train)
    
    y_pred = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))

np.mean(accuracy_scores)

0.7980535999999999

c) Now comes the magic. For each test set instance, generate the predictions of the 1,000 Decision Trees, and keep only the most frequent prediction. This gives you the majority-vote predictions over the test set.

In [7]:
from collections import Counter

def get_majority_vote_for(x):
    x = np.reshape(x, (1, -1))
    counter = Counter()
    for tree in forest:
        prediction = tree.predict(x)[0]
        counter[prediction] += 1
    
    # get most common value predicted
    return counter.most_common()[0][0]

predictions = []
for i in range(len(y_test)):
    predictions.append(get_majority_vote_for(X_test[i, :]))

d) Evaluate these predictions on the test set: you should obtain a slightly higher accuracy than your first model.

In [8]:
final_accuracy = accuracy_score(y_test, predictions)
print('Accuracy obtained with majority vote: {0:.2f}'.format(final_accuracy))

Accuracy obtained with majority vote: 0.86
